**Домашнее задание №1**
*Исполнитель* - Мирахмедова Парвина
*Задача №1*
На основе данных из таблицы track создайте столбчатую диаграмму, показывающую общую сумму минут (длительность треков) в разбивке по жанрам.


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 

query ="""
select g.name as genre_name, 
    sum(t.milliseconds)/60000 as total_duration
    from track t
    join genre g on t.genre_id = g.genre_id
    group by g.name
"""

with set_connection() as conn:
    genre_duration_df = pd.read_sql(query, conn)

fig = px.bar(genre_duration_df, 
             x='genre_name', 
             y='total_duration',
            title="Общая длительность треков по жанрам",
             labels={'genre_name': 'Жанр', 'total_duration': 'Общая длительность (мин)'})

fig.show()


**Задача 2**
На основе данных из таблицы track создайте диаграмму рассеивания, показывающую соотношение минут и мегабайтов.


In [ ]:
query = """
    select milliseconds, bytes
    from track
"""
with set_connection() as conn:
    tracks_df = pd.read_sql(query, conn)

tracks_df['minutes'] = tracks_df['milliseconds'] / 60000
tracks_df['megabytes'] = tracks_df['bytes'] / (1024 * 1024)

fig = px.scatter(tracks_df, x='minutes', y='megabytes', title="Соотношение минут и мегабайтов",
                 labels={'minutes': 'Длительность (мин)', 'megabytes': 'Размер (МБ)'})

fig.show()

**Задача 3**
На основе данных из таблицы invoice создайте коробчатую диаграмму, показывающую распределение дохода (поле total) в разбивке по странам.

In [ ]:

query = """
    select billing_country, total
    from invoice
"""

with set_connection() as conn:
    invoice_df = pd.read_sql(query, conn)

fig = px.box(invoice_df, 
             x='billing_country', 
             y='total', 
             title="Распределение дохода по странам",
             labels={'billing_country': 'Страна', 'total': 'Доход'})

fig.show()


**Задача 4**
На основе данных из таблицы invoice создайте линейчатую диаграмму, показывающую изменение дохода по месяцам за 2025 год.

In [ ]:

query = """
    select invoice_date, total
    from invoice
"""

with set_connection() as conn:
    invoice_df = pd.read_sql(query, conn)

invoice_df['invoice_date'] = pd.to_datetime(invoice_df['invoice_date'], format='%Y-%m-%d %H:%M:%S.%f')

invoice_df['year'] = invoice_df['invoice_date'].dt.year
invoice_df['month'] = invoice_df['invoice_date'].dt.month
invoice_2025_df = invoice_df[invoice_df['year'] == 2025]

monthly_income_df = invoice_2025_df.groupby('month')['total'].sum().reset_index()
monthly_income_df['month'] = monthly_income_df['month'].apply(lambda x: pd.to_datetime(f'2025-{x}-01').strftime('%B'))

fig = px.line(monthly_income_df,
               x='month', 
               y='total',
               title="Изменение дохода по месяцам за 2025 год",
              labels={'month': 'Месяц', 'total': 'Доход'})

fig.show()



**Задача 5**
На основе данных из таблицы invoice, создайте круговую диаграмму, показывающую соотношение продаж в США и в остальном мире. Т.е. ваша круговая диаграмма должна состоять лишь из двух секций: США и остальной мир.

In [ ]:

query = """
    select billing_country, total
    from invoice
"""
with set_connection() as conn:
    invoice_df = pd.read_sql(query, conn)

country_income_df = invoice_df.groupby('billing_country')['total'].sum().reset_index()
country_income_df['region'] = country_income_df['billing_country'].apply(lambda x: 'США' if x == 'USA' else 'Остальной мир')
region_income_df = country_income_df.groupby('region')['total'].sum().reset_index()
fig = px.pie(region_income_df,
             names='region', 
             values='total', 
             title="Соотношение продаж в США и остальном мире")

fig.show()


